In [48]:
# pip install pandas
# pip install sodapy

import pandas as pd
import json
from sodapy import Socrata
import os
import geopandas
import matplotlib.pyplot as plt
from config2 import app_token
from geojson import Feature, FeatureCollection, Point

In [49]:
## PANDAS TO GEOJSON FUNCTION
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [50]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cityofchicago.org", none)

# Example authenticated client (needed for non-public datasets):
# client = Socrata("data.cityofchicago.org",
#                  "du5ugsgl0w1g4ql37x78pjbc5",
#                  username= "dr.harmony",
#                  password= "F3ar&Loathing")

# # Example authenticated client (needed for non-public datasets):
client = Socrata("data.cityofchicago.org",
                 app_token)
#                  username= username,
#                  password= password)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("85ca-t3if", limit = 500000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [51]:
results_df

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,...,crash_date_est_i,photos_taken_i,statements_taken_i,private_property_i,work_zone_i,work_zone_type,workers_present_i,dooring_i,rd_no,lane_cnt
0,3fa1443347f8f5acfbea4d70170b000cdcf7c7cd919a7f...,2020-10-13T00:12:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fa733ebf92caa9413d5a8ed9c1dfc28f9a96869b7782b4...,2020-10-12T23:30:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,d16c88cf5ba23c72e560ce69b4324a9d7ce8776ce7d896...,2020-10-12T22:45:00.000,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3c4b2c7fedbba09e8889d40dd01d2a20144ad8c63c4444...,2020-10-12T22:30:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,079e6cac8d7ad9d88e50988066ed8fd4671bc48d2618f3...,2020-10-12T22:20:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,...,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447083,964aaaeb569e364886cfbdf89ca73e09ca15cd916d87b2...,2014-06-25T19:00:00.000,30,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JC321679,NaN
447084,1d0232afecbdfd01968555aa956a688fd6f55a2bd1984f...,2014-02-24T19:45:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ164689,NaN
447085,957783a4787318f005a7dbc920e4c84cb9ac8aa7329a62...,2014-01-21T07:40:00.000,30,YIELD,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HZ122950,NaN
447086,f62e27317feb174811cf4fefeb9fa1064fea6c0619a873...,2014-01-18T18:14:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JB442550,NaN


In [52]:
results_df.columns

Index(['crash_record_id', 'crash_date', 'posted_speed_limit',
       'traffic_control_device', 'device_condition', 'weather_condition',
       'lighting_condition', 'first_crash_type', 'trafficway_type',
       'alignment', 'roadway_surface_cond', 'road_defect', 'report_type',
       'crash_type', 'damage', 'date_police_notified',
       'prim_contributory_cause', 'sec_contributory_cause', 'street_no',
       'street_direction', 'street_name', 'beat_of_occurrence', 'num_units',
       'most_severe_injury', 'injuries_total', 'injuries_fatal',
       'injuries_incapacitating', 'injuries_non_incapacitating',
       'injuries_reported_not_evident', 'injuries_no_indication',
       'injuries_unknown', 'crash_hour', 'crash_day_of_week', 'crash_month',
       'latitude', 'longitude', 'location', 'hit_and_run_i',
       'intersection_related_i', 'crash_date_est_i', 'photos_taken_i',
       'statements_taken_i', 'private_property_i', 'work_zone_i',
       'work_zone_type', 'workers_present_i', 

In [53]:
crashes = results_df.drop(columns=['crash_record_id', 'location', 'injuries_unknown', 'injuries_incapacitating', 'injuries_no_indication', 'injuries_reported_not_evident', 'injuries_non_incapacitating', 'private_property_i', 'crash_date_est_i', 'statements_taken_i', 'photos_taken_i', 'work_zone_i', 'work_zone_type', 'workers_present_i', 'dooring_i'])
crashes

,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,...,injuries_fatal,crash_hour,crash_day_of_week,crash_month,latitude,longitude,hit_and_run_i,intersection_related_i,rd_no,lane_cnt
0,2020-10-13T00:12:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,DRY,...,0,0,3,10,41.95361776,-87.836539774,NaN,NaN,NaN,NaN
1,2020-10-12T23:30:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,...,0,23,2,10,41.780133017,-87.581620874,Y,NaN,NaN,NaN
2,2020-10-12T22:45:00.000,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,...,0,22,2,10,41.678355568,-87.638164227,Y,NaN,NaN,NaN
3,2020-10-12T22:30:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,UNKNOWN,...,0,22,2,10,41.918800215,-87.746316616,Y,N,NaN,NaN
4,2020-10-12T22:20:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,...,0,22,2,10,41.87496853,-87.740614971,Y,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447083,2014-06-25T19:00:00.000,30,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,WET,...,0,19,4,6,41.835886103,-87.724474013,Y,NaN,JC321679,NaN
447084,2014-02-24T19:45:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,RAIN,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,WET,...,0,19,2,2,41.884016475,-87.701142758,NaN,NaN,HZ164689,NaN
447085,2014-01-21T07:40:00.000,30,YIELD,NO CONTROLS,CLEAR,DAYLIGHT,ANGLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,DRY,...,0,7,3,1,41.760710194,-87.56194603,NaN,NaN,HZ122950,NaN
447086,2014-01-18T18:14:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DUSK,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,DRY,...,0,18,7,1,41.885609917,-87.63875619,NaN,NaN,JB442550,NaN


In [54]:
crashes.rename(columns={"beat_of_occurrence": "Beat"}, inplace = True)

In [55]:
crashes.dtypes

crash_date                 object
posted_speed_limit         object
traffic_control_device     object
device_condition           object
weather_condition          object
lighting_condition         object
first_crash_type           object
trafficway_type            object
alignment                  object
roadway_surface_cond       object
road_defect                object
report_type                object
crash_type                 object
damage                     object
date_police_notified       object
prim_contributory_cause    object
sec_contributory_cause     object
street_no                  object
street_direction           object
street_name                object
Beat                       object
num_units                  object
most_severe_injury         object
injuries_total             object
injuries_fatal             object
crash_hour                 object
crash_day_of_week          object
crash_month                object
latitude                   object
longitude     

In [56]:
crashes[["Beat"]] = crashes[["Beat"]].apply(pd.to_numeric)
crashes.dtypes

crash_date                  object
posted_speed_limit          object
traffic_control_device      object
device_condition            object
weather_condition           object
lighting_condition          object
first_crash_type            object
trafficway_type             object
alignment                   object
roadway_surface_cond        object
road_defect                 object
report_type                 object
crash_type                  object
damage                      object
date_police_notified        object
prim_contributory_cause     object
sec_contributory_cause      object
street_no                   object
street_direction            object
street_name                 object
Beat                       float64
num_units                   object
most_severe_injury          object
injuries_total              object
injuries_fatal              object
crash_hour                  object
crash_day_of_week           object
crash_month                 object
latitude            

In [57]:
# Reference the file where the CSV is located
neighborhood_path = "static/data/ChicagoAreasBeatDistricts.csv"

# Import the data into a Pandas DataFrame
neighborhoods_df = pd.read_csv(neighborhood_path)
neighborhoods_df.dtypes

Beat              int64
District          int64
Community Area    int64
dtype: object

In [58]:
crash_neighborhood_merge = pd.merge(left=crashes, right=neighborhoods_df, how='right', on=['Beat'])
# crash_neighborhood_merge = pd.concat([crashes, neighborhoods_df], axis=1)
crash_neighborhood_merge_clean = crash_neighborhood_merge[['crash_date', 'posted_speed_limit', 'traffic_control_device',
       'device_condition', 'weather_condition', 'lighting_condition',
       'first_crash_type', 'trafficway_type', 'alignment', 'injuries_fatal',
       'roadway_surface_cond', 'road_defect', 'report_type', 'crash_type',
       'intersection_related_i', 'damage', 'date_police_notified',
       'prim_contributory_cause', 'sec_contributory_cause', 'street_no',
       'street_direction', 'street_name', 'District', 'Community Area', 'Beat', 'num_units',
       'most_severe_injury', 'injuries_total', 'crash_hour',
       'crash_day_of_week', 'crash_month', 'latitude', 'longitude',
       'hit_and_run_i', 'rd_no', 'lane_cnt']]
crash_neighborhood_merge_clean

,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,injuries_fatal,...,most_severe_injury,injuries_total,crash_hour,crash_day_of_week,crash_month,latitude,longitude,hit_and_run_i,rd_no,lane_cnt
0,2020-10-13T00:12:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,0,...,NONINCAPACITATING INJURY,1,0,3,10,41.95361776,-87.836539774,NaN,NaN,NaN
1,2020-10-08T16:20:00.000,35,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,CENTER TURN LANE,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,16,5,10,41.952172091,-87.835790333,NaN,NaN,NaN
2,2020-10-07T15:15:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,OTHER,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,15,4,10,41.959252309,-87.836590522,NaN,NaN,NaN
3,2020-10-07T00:01:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,UNKNOWN,PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,0,4,10,41.975190437,-87.845349712,Y,NaN,NaN
4,2020-10-06T19:47:00.000,10,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,PARKING LOT,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,19,3,10,41.96652597,-87.836829397,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866458,2018-08-09T17:45:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,REAR END,RAMP,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,17,5,8,NaN,NaN,NaN,JB387944,4
866459,2018-08-02T21:45:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,21,5,8,NaN,NaN,NaN,JB377421,NaN
866460,2018-07-16T05:40:00.000,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,ONE-WAY,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,5,2,7,NaN,NaN,NaN,JB351498,NaN
866461,2018-07-10T07:10:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,OTHER,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,7,3,7,NaN,NaN,NaN,JB343207,2


In [59]:
crash_neighborhood_merge_clean.columns

Index(['crash_date', 'posted_speed_limit', 'traffic_control_device',
       'device_condition', 'weather_condition', 'lighting_condition',
       'first_crash_type', 'trafficway_type', 'alignment', 'injuries_fatal',
       'roadway_surface_cond', 'road_defect', 'report_type', 'crash_type',
       'intersection_related_i', 'damage', 'date_police_notified',
       'prim_contributory_cause', 'sec_contributory_cause', 'street_no',
       'street_direction', 'street_name', 'District', 'Community Area', 'Beat',
       'num_units', 'most_severe_injury', 'injuries_total', 'crash_hour',
       'crash_day_of_week', 'crash_month', 'latitude', 'longitude',
       'hit_and_run_i', 'rd_no', 'lane_cnt'],
      dtype='object')

In [60]:
crash_neighborhood_merge_clean

,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,injuries_fatal,...,most_severe_injury,injuries_total,crash_hour,crash_day_of_week,crash_month,latitude,longitude,hit_and_run_i,rd_no,lane_cnt
0,2020-10-13T00:12:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,0,...,NONINCAPACITATING INJURY,1,0,3,10,41.95361776,-87.836539774,NaN,NaN,NaN
1,2020-10-08T16:20:00.000,35,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,CENTER TURN LANE,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,16,5,10,41.952172091,-87.835790333,NaN,NaN,NaN
2,2020-10-07T15:15:00.000,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,OTHER,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,15,4,10,41.959252309,-87.836590522,NaN,NaN,NaN
3,2020-10-07T00:01:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,UNKNOWN,PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,0,4,10,41.975190437,-87.845349712,Y,NaN,NaN
4,2020-10-06T19:47:00.000,10,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,PARKING LOT,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,19,3,10,41.96652597,-87.836829397,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866458,2018-08-09T17:45:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,REAR END,RAMP,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,17,5,8,NaN,NaN,NaN,JB387944,4
866459,2018-08-02T21:45:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,21,5,8,NaN,NaN,NaN,JB377421,NaN
866460,2018-07-16T05:40:00.000,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,ONE-WAY,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,5,2,7,NaN,NaN,NaN,JB351498,NaN
866461,2018-07-10T07:10:00.000,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,OTHER,STRAIGHT AND LEVEL,0,...,NO INDICATION OF INJURY,0,7,3,7,NaN,NaN,NaN,JB343207,2


In [61]:
## DROPPING COLUMNS
crashes_in_areas = crash_neighborhood_merge_clean.drop(columns=['posted_speed_limit', 'traffic_control_device', 'crash_type', 'device_condition', 'weather_condition', 'lighting_condition', 'trafficway_type', 'alignment',
'roadway_surface_cond', 'road_defect', 'report_type','intersection_related_i', 'damage', 'date_police_notified',
'prim_contributory_cause', 'sec_contributory_cause', 'street_no',
'street_direction', 'street_name', 'District', 'num_units', 'most_severe_injury', 'crash_hour',
'crash_day_of_week', 'crash_month', 'rd_no', 'lane_cnt', 'hit_and_run_i'])

In [62]:
crashes_in_areas

,crash_date,first_crash_type,injuries_fatal,Community Area,Beat,injuries_total,latitude,longitude
0,2020-10-13T00:12:00.000,PARKED MOTOR VEHICLE,0,10,1614.0,1,41.95361776,-87.836539774
1,2020-10-08T16:20:00.000,REAR END,0,10,1614.0,0,41.952172091,-87.835790333
2,2020-10-07T15:15:00.000,REAR END,0,10,1614.0,0,41.959252309,-87.836590522
3,2020-10-07T00:01:00.000,PARKED MOTOR VEHICLE,0,10,1614.0,0,41.975190437,-87.845349712
4,2020-10-06T19:47:00.000,REAR END,0,10,1614.0,0,41.96652597,-87.836829397
...,...,...,...,...,...,...,...,...
866458,2018-08-09T17:45:00.000,REAR END,0,76,1652.0,0,NaN,NaN
866459,2018-08-02T21:45:00.000,SIDESWIPE SAME DIRECTION,0,76,1652.0,0,NaN,NaN
866460,2018-07-16T05:40:00.000,SIDESWIPE SAME DIRECTION,0,76,1652.0,0,NaN,NaN
866461,2018-07-10T07:10:00.000,SIDESWIPE SAME DIRECTION,0,76,1652.0,0,NaN,NaN


In [63]:
## DROP any row with NaN values ** THIS REMOVES SOME CRASHES BUT KEEPS HIGH QUALITY RECORDED **
df = crashes_in_areas.dropna()
df

,crash_date,first_crash_type,injuries_fatal,Community Area,Beat,injuries_total,latitude,longitude
0,2020-10-13T00:12:00.000,PARKED MOTOR VEHICLE,0,10,1614.0,1,41.95361776,-87.836539774
1,2020-10-08T16:20:00.000,REAR END,0,10,1614.0,0,41.952172091,-87.835790333
2,2020-10-07T15:15:00.000,REAR END,0,10,1614.0,0,41.959252309,-87.836590522
3,2020-10-07T00:01:00.000,PARKED MOTOR VEHICLE,0,10,1614.0,0,41.975190437,-87.845349712
4,2020-10-06T19:47:00.000,REAR END,0,10,1614.0,0,41.96652597,-87.836829397
...,...,...,...,...,...,...,...,...
866384,2015-11-07T09:30:00.000,PARKED MOTOR VEHICLE,0,27,1124.0,0,41.878797755,-87.704072334
866385,2015-10-09T16:33:00.000,REAR END,0,27,1124.0,0,41.877428045,-87.707249275
866386,2015-10-08T17:30:00.000,REAR END,0,27,1124.0,0,41.880995041,-87.701059473
866387,2015-09-14T17:30:00.000,TURNING,0,27,1124.0,0,41.876089738,-87.696217119


In [64]:
## rename Community Area and Beat columns names
df = df.rename(columns = {"Community Area": "communityArea", "Beat": "beat"})
df

,crash_date,first_crash_type,injuries_fatal,communityArea,beat,injuries_total,latitude,longitude
0,2020-10-13T00:12:00.000,PARKED MOTOR VEHICLE,0,10,1614.0,1,41.95361776,-87.836539774
1,2020-10-08T16:20:00.000,REAR END,0,10,1614.0,0,41.952172091,-87.835790333
2,2020-10-07T15:15:00.000,REAR END,0,10,1614.0,0,41.959252309,-87.836590522
3,2020-10-07T00:01:00.000,PARKED MOTOR VEHICLE,0,10,1614.0,0,41.975190437,-87.845349712
4,2020-10-06T19:47:00.000,REAR END,0,10,1614.0,0,41.96652597,-87.836829397
...,...,...,...,...,...,...,...,...
866384,2015-11-07T09:30:00.000,PARKED MOTOR VEHICLE,0,27,1124.0,0,41.878797755,-87.704072334
866385,2015-10-09T16:33:00.000,REAR END,0,27,1124.0,0,41.877428045,-87.707249275
866386,2015-10-08T17:30:00.000,REAR END,0,27,1124.0,0,41.880995041,-87.701059473
866387,2015-09-14T17:30:00.000,TURNING,0,27,1124.0,0,41.876089738,-87.696217119


In [66]:
# CREATING GEOJSON FILE FOR EXPORT
# cols = ['crash_date', 'first_crash_type', 'injuries_fatal', 'communityArea', 'beat', 'injuries_total', 'latitude', 'longitude']
# geojson = df_to_geojson(df, cols)

## USING THE GEOPANDAS
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.longitude, df.latitude))


In [67]:
print(gdf.head())

                crash_date      first_crash_type injuries_fatal  \
0  2020-10-13T00:12:00.000  PARKED MOTOR VEHICLE              0   
1  2020-10-08T16:20:00.000              REAR END              0   
2  2020-10-07T15:15:00.000              REAR END              0   
3  2020-10-07T00:01:00.000  PARKED MOTOR VEHICLE              0   
4  2020-10-06T19:47:00.000              REAR END              0   

   communityArea    beat injuries_total      latitude      longitude  \
0             10  1614.0              1   41.95361776  -87.836539774   
1             10  1614.0              0  41.952172091  -87.835790333   
2             10  1614.0              0  41.959252309  -87.836590522   
3             10  1614.0              0  41.975190437  -87.845349712   
4             10  1614.0              0   41.96652597  -87.836829397   

                     geometry  
0  POINT (-87.83654 41.95362)  
1  POINT (-87.83579 41.95217)  
2  POINT (-87.83659 41.95925)  
3  POINT (-87.84535 41.97519)  
4  P

In [68]:
# gdf.to_file("crashes_in_areas.geojson", driver='GeoJSON')